In [8]:
import os
import numpy as np
import PIL.Image as Image
import matplotlib.pyplot as plt
import tensorflow as tf


def show_image_from_array(image_array, cmap='gray'):
    plt.imshow(image_array, cmap=cmap)
    plt.axis('off')


def load_images_and_labels(directory, target_size=(100, 100)):
    images = []
    labels = []

    # Iterate through each subdirectory (assuming each folder corresponds to a label)
    for label_name in os.listdir(directory):
        folder_path = os.path.join(directory, label_name)

        # Check if it's a directory
        if os.path.isdir(folder_path):
            # Iterate through each image file in the folder
            for filename in os.listdir(folder_path):
                image_path = os.path.join(folder_path, filename)
                try:
                    # Open image using PIL
                    with Image.open(image_path) as img:
                        # Resize image
                        img = img.resize(target_size)
                        # Convert image to array
                        images.append(np.array(img))
                        labels.append(label_name)
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")

    return images, labels



# Example usage
directory_path = "trainData"
images, labels = load_images_and_labels(directory_path,(100,100))

#label inversion in to integer array
labels = np.array(labels)
labels[labels == "healthy"] = 0
labels[labels == "infected"] = 1
labels = [int(num_str) for num_str in labels]


#image reshaping
images = np.array(images)/255



#cnn mideling

# model2 = tf.keras.Sequential([
#     #cnn layer
#     tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation = 'relu',input_shape=(150,150,4)),
#     tf.keras.layers.MaxPool2D(2,2),
    
#     #layer 2
#     tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation = 'relu'),
#     tf.keras.layers.MaxPool2D(2,2),
    
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),

#     #dense layer
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64,activation="relu"),
#     tf.keras.layers.Dense(10,activation="softmax") #output layer  
# ])

model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 4)), # Assuming input images are 100x100 pixels with 4 channels
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])


model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


#model2.summary()

train_dataset = tf.data.Dataset.from_tensor_slices((images,labels)).batch(32)


# #train the model
model2.fit(
    train_dataset,
    epochs=30,   #iteration over the model to predict correct
    validation_data = train_dataset
)



# Evaluate the model
test_loss, test_accuracy = model2.evaluate(train_dataset)
print("Test Accuracy:", test_accuracy)


Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 969ms/step - accuracy: 0.0300 - loss: 2.8156 - val_accuracy: 0.5000 - val_loss: 0.7822
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 474ms/step - accuracy: 0.4896 - loss: 1.0104 - val_accuracy: 0.5000 - val_loss: 0.8569
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 468ms/step - accuracy: 0.4896 - loss: 1.0035 - val_accuracy: 0.5000 - val_loss: 0.7053
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 524ms/step - accuracy: 0.4896 - loss: 0.9160 - val_accuracy: 0.5000 - val_loss: 0.5632
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 459ms/step - accuracy: 0.4896 - loss: 0.7464 - val_accuracy: 0.5000 - val_loss: 0.5234
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 447ms/step - accuracy: 0.4896 - loss: 0.6263 - val_accuracy: 0.5000 - val_loss: 0.5041
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 484ms/step - accuracy: 0.4896 - loss: 0.6325 - val_accuracy: 0.5000 - val_loss: 0.4724
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 496ms/step - accuracy: 0.4896 - loss: 0.6146 - val_accuracy: 0.6471 - val_loss:

In [32]:
#load image
image_path = "testImg2.png"
image = Image.open(image_path)
image = image.resize((100, 100))  # Resize the image to match the input shape of the model
image = np.array(image) / 255.0  # Normalize pixel values

# Add an extra dimension to match the input shape expected by the model
image = np.expand_dims(image, axis=0)  # Shape: (1, 100, 100, 4)


predict = model2.predict(image)
predicted_class = np.argmax(predict)

print("0 = healthy 1 = infected")
print("prediction:",predicted_class)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
0 = healthy 1 = infected
prediction: 0
